<a href="https://colab.research.google.com/github/anunknownpleasure/Pricing-assets-with-deep-learning/blob/main/Asset_pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Installing libraries


In [1]:
!pip install getFamaFrenchFactors

In [2]:
!pip install lxml

In [3]:
!pip install pandas_datareader

# 2. Data Import and preprocessing

## 2a. Importing Fama-French 5 factor data


In [109]:
import numpy as np
import pandas as pd
import getFamaFrenchFactors

print("Successfully imported getFamaFrenchFactors!")
print(dir(getFamaFrenchFactors))

Successfully imported getFamaFrenchFactors!
['BeautifulSoup', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__warningregistry__', 'all_factor_links', 'all_factors_text', 'bold_tags', 'carhart4Factor', 'csv_links', 'factor_dict', 'famaFrench3Factor', 'famaFrench5Factor', 'ff3factor_dict', 'home_url', 'link', 'links_for_factor', 'momAndOthers_dict', 'momentumFactor', 'pd', 'relativedelta', 'requests', 'response', 'sib', 'soup', 'text', 'text_to_search', 'txt_links', 'url']


In [111]:
from getFamaFrenchFactors import famaFrench5Factor


# Get the factors
factors_df = famaFrench5Factor()

# Adjust from percentage
factors_df[['Mkt-RF', 'SMB', 'HML', 'RF']] = factors_df[['Mkt-RF', 'SMB', 'HML', 'RF']] / 100



In [73]:
factors_df.head(20)

,date_ff_factors,Mkt-RF,SMB,HML,RMW,CMA,RF
0,1963-07-31,-0.000039,-0.000048,-0.000081,0.0064,-0.0115,0.000027
1,1963-08-31,0.000508,-0.000080,0.000170,0.0040,-0.0038,0.000025
2,1963-09-30,-0.000157,-0.000043,0.000000,-0.0078,0.0015,0.000027
3,1963-10-31,0.000254,-0.000134,-0.000004,0.0279,-0.0225,0.000029
4,1963-11-30,-0.000086,-0.000085,0.000173,-0.0043,0.0227,0.000027
5,1963-12-31,0.000183,-0.000189,-0.000021,0.0012,-0.0025,0.000029
6,1964-01-31,0.000227,0.000010,0.000163,0.0021,0.0148,0.000030
7,1964-02-29,0.000155,0.000033,0.000281,0.0011,0.0081,0.000026
8,1964-03-31,0.000141,0.000141,0.000329,-0.0203,0.0298,0.000031
9,1964-04-30,0.000011,-0.000148,-0.000054,-0.0132,-0.0113,0.000029


We adjust the start date to -1-31-1964

In [112]:
FFdata = factors_df.iloc[6:]
FFdata['date_ff_factors'] = pd.to_datetime(FFdata['date_ff_factors'])
FFdata = FFdata.set_index('date_ff_factors')

/tmp/ipython-input-4197135584.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FFdata['date_ff_factors'] = pd.to_datetime(FFdata['date_ff_factors'])


## 2b. Importing the macroeconomic data

In [113]:
import pandas_datareader.data as web

# --- 1. Define the 5 Long-History Macro Indicators (FRED Tickers) ---
long_macro_tickers = {
    'Term_Spread': 'T10YFFM',      # 10-Yr Yield minus Fed Funds Rate
    'Default_Spread': 'AAAFFM',     # Baa Corp Yield minus Aaa Corp Yield (Risk Aversion Proxy)
    'Ind_Production': 'INDPRO',     # Industrial Production Index
    'Unemployment': 'UNRATE',       # Civilian Unemployment Rate
    'Consumer_Sentiment': 'UMCSENT' # University of Michigan Consumer Sentiment
}

# --- 2. Define the Time Period
start_date = '1964-01-01'
end_date = pd.to_datetime('today').strftime('%Y-%m-%d')

# --- 3. Fetch Data from FRED ---
try:
    macro_data = web.DataReader(
        list(long_macro_tickers.values()),
        'fred',
        start=start_date,
        end=end_date
    )
    macro_data.columns = list(long_macro_tickers.keys())

except Exception as e:
    print(f"Error fetching data from FRED: {e}")
    macro_data = pd.DataFrame(index=pd.date_range(start_date, end_date, freq='M'))

# --- 4. Align Data to Monthly Frequency ---

# 4a. Forward-Fill any monthly gaps (common in macro data)
macro_data = macro_data.ffill()

# 4b. Ensure all data points are at the end of the month for clean alignment
macro_data = macro_data.resample('ME').last()


# --- 5. Display the result ---
print(f"Macro Data Imported and Aligned ({len(macro_data)} periods, starting {macro_data.index.min().strftime('%Y-%m')}):")
print(macro_data.head())
print("\n... and the tail:")
print(macro_data.tail())

# The resulting 'macro_data' DataFrame is ready for merging with Fama-French data.

Macro Data Imported and Aligned (741 periods, starting 1964-01):
            Term_Spread  Default_Spread  Ind_Production  Unemployment  \
DATE                                                                    
1964-01-31         0.69            0.91         27.7409           5.6   
1964-02-29         0.67            0.88         27.9291           5.4   
1964-03-31         0.79            0.95         27.9291           5.4   
1964-04-30         0.76            0.93         28.3861           5.3   
1964-05-31         0.70            0.91         28.5474           5.1   

            Consumer_Sentiment  
DATE                            
1964-01-31                 NaN  
1964-02-29                99.5  
1964-03-31                99.5  
1964-04-30                99.5  
1964-05-31                98.5  

... and the tail:
            Term_Spread  Default_Spread  Ind_Production  Unemployment  \
DATE                                                                    
2025-05-31         0.09    

In [9]:
macro_data.columns

Index(['Term_Spread', 'Default_Spread', 'Ind_Production', 'Unemployment',
       'Consumer_Sentiment'],
      dtype='object')

##2c. Importing the FF-portfolios

In [106]:
# --- 1. Define Time Period ---
# Must match the start date used for your FF factors and macro data (e.g., 1964-01-01)
start_date = '1964-01-01'
end_date = pd.to_datetime('today').strftime('%Y-%m-%d')

# --- 2. Fetch the 25 Portfolios (Size x Book-to-Market) ---
# The data is downloaded as a dictionary object
ff_portfolio = web.DataReader(
    '25_Portfolios_5x5',
    'famafrench',
    start=start_date,
    end=end_date
)

df_returns_25 = ff_portfolio[0]
print(df_returns_25.shape)
df_returns_25 = df_returns_25.replace([-99.99, -999], np.nan) # Missing values are indicated by -99.99 or -999
df_returns_25.dropna(inplace=True)
print(df_returns_25.shape)
df_returns_25 = (df_returns_25/100)# Converting from percentage to fraction

# Convert PeriodIndex to DatetimeIndex at the end of the month
df_returns_25.index = df_returns_25.index.to_timestamp(how = 'end').date




/tmp/ipython-input-3823234249.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_portfolio = web.DataReader(
/tmp/ipython-input-3823234249.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_portfolio = web.DataReader(
/tmp/ipython-input-3823234249.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_portfolio = web.DataReader(
/tmp/ipython-input-3823234249.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'obje

(740, 25)
(740, 25)


## 2d. Combining all the data into a DataFrame

In [120]:

# Making the index into a datetime object
macro_data.index = pd.to_datetime(macro_data.index)


# Combine the dataframes using merge on the index

combined_data_FF_macro = pd.merge(FFdata, macro_data, left_index=True, right_index=True, how='inner') # Combining FF and Macro. Dropping first row because of a null value

combined_data = pd.merge(combined_data_FF_macro, df_returns_25, left_index=True, right_index=True, how='inner')

combined_data.dropna(inplace=True) # Dropping NaN entries

# Display the combined data
print("Combined Data:")
display(combined_data.head())
print("\n... and the tail:")
display(combined_data.tail())

Combined Data:


,Mkt-RF,SMB,HML,RMW,CMA,RF,Term_Spread,Default_Spread,Ind_Production,Unemployment,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
1964-02-29,0.000155,0.000033,0.000281,0.0011,0.0081,0.000026,0.67,0.88,27.9291,5.4,...,0.025943,0.015619,0.028444,0.072047,0.046121,0.018271,0.005232,0.010194,0.039989,0.037567
1964-03-31,0.000141,0.000141,0.000329,-0.0203,0.0298,0.000031,0.79,0.95,27.9291,5.4,...,0.017750,0.029767,0.052497,0.071287,0.007247,0.011575,0.007635,0.036237,0.038382,0.001491
1964-04-30,0.000011,-0.000148,-0.000054,-0.0132,-0.0113,0.000029,0.76,0.93,28.3861,5.3,...,-0.027045,0.003434,0.019784,-0.026384,-0.022805,0.002272,0.014745,0.008082,-0.009054,0.024147
1964-05-31,0.000141,-0.000062,0.000181,-0.0015,0.0013,0.000026,0.70,0.91,28.5474,5.1,...,0.011914,0.022992,0.013559,0.013281,0.040990,0.020599,0.003304,0.011776,0.042859,0.033968
1964-06-30,0.000127,0.000013,0.000068,-0.0033,0.0010,0.000030,0.67,0.91,28.6280,5.2,...,0.010927,0.014771,0.011035,0.024965,0.031119,0.009744,0.028644,0.004437,0.013682,0.024217



... and the tail:


,Mkt-RF,SMB,HML,RMW,CMA,RF,Term_Spread,Default_Spread,Ind_Production,Unemployment,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
2025-04-30,-0.000084,-0.000186,-0.000340,-0.0285,-0.0267,0.000035,-0.05,1.12,103.6224,4.2,...,-0.008766,-0.012699,-0.020146,-0.039276,-0.072668,0.014106,-0.030129,-0.073867,-0.013472,-0.027941
2025-05-31,0.000606,-0.000072,-0.000288,0.0126,0.0251,0.000038,0.09,1.21,103.6570,4.2,...,0.062577,0.050222,0.035353,0.081175,0.065826,0.078077,0.061296,0.018407,0.026156,0.065684
2025-06-30,0.000486,-0.000002,-0.000160,-0.0319,0.0145,0.000034,0.05,1.13,104.2115,4.1,...,0.020351,0.043192,0.024175,0.073815,0.058024,0.055279,0.062451,0.047405,0.036424,0.070109
2025-07-31,0.000198,-0.000015,-0.000127,-0.0029,-0.0207,0.000034,0.06,1.12,103.8194,4.2,...,0.034206,0.021766,0.009013,-0.001069,-0.019303,0.032949,0.014068,0.012224,0.002333,-0.013744
2025-08-31,0.000185,0.000488,0.000441,-0.0069,0.0207,0.000038,-0.07,1.02,103.9203,4.3,...,0.036524,0.016164,0.023481,0.058848,0.071342,0.011610,0.011927,0.030567,0.054257,0.090799


In [121]:
combined_data.columns

Index(['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Term_Spread',
       'Default_Spread', 'Ind_Production', 'Unemployment',
       'Consumer_Sentiment', 'SMALL LoBM', 'ME1 BM2', 'ME1 BM3', 'ME1 BM4',
       'SMALL HiBM', 'ME2 BM1', 'ME2 BM2', 'ME2 BM3', 'ME2 BM4', 'ME2 BM5',
       'ME3 BM1', 'ME3 BM2', 'ME3 BM3', 'ME3 BM4', 'ME3 BM5', 'ME4 BM1',
       'ME4 BM2', 'ME4 BM3', 'ME4 BM4', 'ME4 BM5', 'BIG LoBM', 'ME5 BM2',
       'ME5 BM3', 'ME5 BM4', 'BIG HiBM'],
      dtype='object')

In [122]:
FF_columns = combined_data.columns[:6]
macro_columns = combined_data.columns[6:11]
portfolio_columns = combined_data.columns[11:]

no_of_FF_features = len(FF_columns)
no_macro_features = len(macro_columns)
no_of_portfolios = len(portfolio_columns)




## 2e. Scaling the data

In [118]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


processed_data = scaler.fit_transform(combined_data)
processed_data[:5]


array([[ 0.37577208, -0.0305934 ,  0.45132359, -0.02751812,  0.59441065,
        -0.24480283, -0.16783942, -0.56927637, -1.76568317, -0.15534895,
                nan,  0.40233574,  0.249364  ,  0.50501711,  0.46543608,
         0.55348369, -0.40896576, -0.26751868,  0.43228184,  0.05681002,
         0.45274812, -0.12360391, -0.1084553 ,  0.31780515,  0.51165303,
         0.31281853, -0.24992634,  0.16272484,  0.03274254,  0.47154545,
         0.36253978,  0.44700266,  0.51198141,  0.16900294,  0.23200703,
         0.09744748],
       [ 0.21503462,  0.04504571,  0.8471173 , -0.07253008,  0.26973421,
        -0.39693727, -0.17997141, -0.58464314, -1.7581049 , -0.2700205 ,
         1.12310801,  0.27598422,  0.24163439,  0.19067469, -0.16787125,
         0.47786754,  0.09867669,  0.09644605,  0.24906577,  0.16478231,
         0.7600242 , -0.11989002,  0.3398467 ,  0.10213541,  0.49003423,
         0.44709266,  0.2709801 ,  0.10622468,  0.35262579,  1.18438658,
         0.5814077 ,  0.18349

We create rolling windows for LSTM input using the macroeconomic variables